# Modeling Demand for Cars with Inverse Product Differentiation Logit

In this notebook, we will explore the dataset used in
Brownstone and Train (1999). We will estimate the Inverse Product Differentiation Logit
model using the FKN estimation routine given the available data using the functions defined below.


In [ ]:
import numpy as np
import pandas as pd 
import os
from numpy import linalg as la
from scipy import optimize
from IPython import display
from matplotlib import pyplot as plt
import itertools as iter

# Files
import Logit_file as logit

Data
====

The data consists of a survey of households regarding their preferences
for car purchase. Each household was given 6 options, but the
characteristics that the respondents were asked about was varied. The
surveys were originally conducted in order to illicit consumer
preferences for alternative-fuel vehicles. The data is *stated
preferences*, in the sense that consumers did not actually buy but just
stated what they would hypothetically choose, which is of course a
drawback. This is very common in marketing when historic data is either
not available or does not provide satisfactory variation. The advantage
of the stated preference data is therefore that the choice set can be
varied greatly (for example, the characteristics includes the
availability of recharging stations, which is important for purchase of
electric cars).

The data has $N=4654$ respondents with $J=6$ cars to choose
from.

Loading the dataset, `car_data.csv`, we get a dataframe with 
$NJ = 27,924$ rows. The column `person_id` runs through $0,1,...,N-1$, and
the column `j` is the index for the car, $\{0,1,...,5\}$. The variable 
`binary_choice` is a dummy, =1 for the car chosen by the respondent. 
A conveneint other variable, `y`, is the index for that car, repeated 
and identical for all $J$ rows for each person. The x-variables describe 
the characteristics of the 6 cars that the respondent was asked to choose 
from. 

We also read in the dataset `car_labels.csv`, which contains the 
variable labels and descriptions for all the variables. 
The list `x_vars` will be used throughout as the list of 
explanatory variables we want to work with. 

In order to get the data into a 3-dimensional array, we access 
the underlying numpy arrays and resize them. For example 

> `x = dat[x_vars].values.resize((N,J,K))`

Note that this will only work because the data is sorted according to 
first `person_id` and then `j`. 

In [ ]:
# Load dataset and variable names
input_path = os.getcwd() # Assigns input path as current working directory (cwd)
dat = pd.read_csv(os.path.join(input_path, 'car_data.csv'))
lab = pd.read_csv(os.path.join(input_path, 'car_labels.csv'), index_col = 'variable')

In [ ]:
display.Image('brownstone_train_tab_1.PNG')

Table 1 from 'Forecasting new product penetration with flexible substitution patterns (1999), D. Brownstone, K. Train'

## Scaling variables

To be consistent with the interpretation of estimates in 'Brownstone & Train (1999)' we rescale some of the explanatory variables. Furthermore, Logit models are most stable numerically if we ensure that variables are scaled near to $\pm 1$. 

In [ ]:
dat['range'] = dat['range'] / 100                  # Hundreds of miles that the vehicle can travel between fuelings
dat['top_speed'] = dat['top_speed'] / 100          # Highest speed that the vehicle can attain, in hundreds of miles per hour
dat['size'] = dat['size'] / 10                     # Scaled categorical variable for numerical purposes
dat['acceleration'] = dat['acceleration'] / 10     # Measured in tens of seconds
dat['operating_cost'] = dat['operating_cost'] / 10 # Measured in tens of cents per mile

Since, respectively, 'EV' and'Non-EV' and also 'CNG' and 'Non-CNG' are equivalent we exclude the latter and keep all the other characteristics as explanatory variables.  

In [ ]:
# variables to use as explanatory variables
x_vars = list(lab.iloc[3:-4].index.values) # variable names

In [ ]:
# dimensions of data
N = dat.person_id.nunique()
J = dat.j.nunique()
K = len(x_vars)

Finally, we will primarily use numpy data types and numpy functions in this notebook. Hence we store our response variable 'y' and our explanatory variables 'x' as numpy arrays.

In [ ]:
# response and explanatory variables as numpy arrays
a = dat['y'].values.reshape((N,J))
a = a[:, 0] # All values are equal along axis=1. Becomes an (N,) array i.e. it is a vector.
y = pd.get_dummies(a).to_numpy() # Convert y to an (N,J) array as the onehot encoding
x = dat[x_vars].values.reshape((N,J,K))

#### Multinomial Logit - for comparison
Estimating a Logit model via maximum likelihood with an initial guess of parameters $\hat \beta^0 = 0$ yields estimated parameters $\hat \beta^{\text{logit}}$ given as...

In [ ]:
beta_0 = np.zeros((K,))

# Estimate the model
res_logit = logit.estimate_logit(logit.q_logit, beta_0, y, x)

In [ ]:
logit_beta = np.round(res_logit['beta'], decimals=3)
logit_p = res_logit['p']
pd.DataFrame({'beta' : [str(logit_beta[i]) + '***' if logit_p[i] < 0.01 else str(logit_beta[i]) + '**' if logit_p[i] < 0.05 else str(logit_beta[i]) + '*' if logit_p[i] < 0.1 else str(logit_beta[i]) for i in range(len(logit_beta))], 
              'se': np.round(res_logit['se'], decimals=3), 
              't (beta = 0)' : np.round(res_logit['t'], decimals=3), 
              'p' : np.round(res_logit['p'], decimals=3)}, index=x_vars).rename_axis(columns='variables')

We then compute the corresponding Logit choice probabilities

In [ ]:
logit_q = logit.logit_ccp(logit_beta, x)

We also find the elasticities and diversion ratios implied by the logit model as follows...

In [ ]:
epsilon_logit = logit.logit_elasticity(logit_q, logit_beta, 0) # Elasticities wrt. the price-to-log-income characteristic
DR_logit_hat = logit.logit_diversion_ratio(logit_q, logit_beta)

## IPDL Model

### Nests of cars
In our case, we construct nests along the dimensions of 'Fuel Type' and 'Body Type' as suggested by the figure below. Each alternative is assigned to two nests: One representing fuel type (Electric, Combustion/Natural Gas , Methanol) and one representing body type (Car, Truck/Van).

display.Image('brownstone_train_fig_1.gif')

Figure 1 from 'Forecasting new product penetration with flexible substitution patterns (1999), D. Brownstone, K. Train'

# The IPDL model - Nesting structure

The IPDL model is a generalization of the nested logit model where each alternative may belong to more than one nest. Before fully introducing the model, we construct the nesting structure.


## Constructing nests

Let $\Delta=\left\{q\in \mathbb{R}^J_+: \sum_{j=1}^J q_j=1\right\}$ denote the probability simplex. For each group of nests $g=1,\ldots, G$, nest membership is denoted by the matrix $\Psi^g\in \mathbb R^{C_g\times J}$: $\Psi^g_{cj}=1$ if product $j$ belongs to nest $c$ and zero otherwise, and each product can only belong to one nest within each group, meaning that $\sum_{c=1}^{C_g}\Psi^g_{cj}=1$ for all $j$ and all $g$. The matrix-vector product $\Psi^gq$ is then
$$
\Psi^g q=\sum_j \Psi^{g}_{cj}q_j=\left(\begin{array}{c}
\sum_{j:\Psi^g_{1j}=1} q_j \\
\vdots \\
\sum_{j: \Psi^g_{C_gj}=1}q_j
\end{array}\right),
$$
and the vector $\Psi^gq$ is a vector of nest-specific choice probabilities, i.e. the sum of the probabilities within each nest.

## The perturbation function $\Omega$

In the following, a vector $z\in \mathbb R^d$ is always a column vector. We now construct the IPDL perturbation function which has the form (where for a vector $z$, the logarithm is applied elementwise and $z'$ denote the transpose)
$$
\Omega(q|\lambda)= (1-\sum_{g=1}^G \lambda_g) q'\ln q +\sum_{g=1}^{G} \lambda_g \left(\Psi^g q \right)'\ln \left(\Psi^g q\right).
$$
Note that since $\Psi^g q$ denotes a probability distribution over the nests, the term $(\Psi^gq)'\ln (\Psi^gq)$ is the (negative) entropy of the probability distribution $\Psi^g q$. Note also that as each nest has at least one member, and $q$ is strictly positive, $\Psi^gq$ is also strictly positive. When the parameters $\lambda_g$ satisfy $\lambda_g>0$ and
$$
\sum_g \lambda_g<1,
$$
the function $\Omega(\cdot|\lambda)$ is a strictly convex function of $q$, and the utility maximization problem has a unique interior (meaning strictly positive choice probabilities) solution. When there is only one group of nests, $G=1$, then $\Omega$ induces the nested logit choice probabilities (note though that the nested logit model is often parameterized in terms of the nesting parameter $\mu=1-\lambda$ instead!). 

It will be convenient to define a choice probability function for a given vector of payoffs $u$ as
$$
P(u|\lambda)=\arg \max_{q\in \Delta}q'u-\Omega(q|\lambda).
$$
Letting $\theta$ denote the full vector of parameters, $\theta=(\beta',\lambda')'$, the individual choice probabilities is a function of the matrix $\mathbf{X}_i$ and the parameters $\theta$, as
$$
p(\mathbf{X}_i,\theta)=\arg\max_{q\in \Delta}q'\mathbf{X}_i \beta-(1-\sum_{g=1}^G\lambda_g)q'\ln q-\sum_{g=1}^G\lambda_g \left(\Psi^g q \right)'\ln \left(\Psi^g q\right)
$$

In [ ]:
def Create_incidence_matrix(allocation, x):

    '''
    This function creates the incidence matrices \Psi^g that partition products into nest groupings and nests.
    
    Args.
        allocation: a list of lists with tuples of products j zipped according to their nests in each nest grouping
        x: a matrix of covariates
    
    Output
        psi: a dictionary of the matrices \psi^g as columns
        nests: a numpy array of the amount nests in each nest grouping, i.e. the C_g's
    '''
    
    import numpy as np

    N,J,K = x.shape
    num_groups = len(allocation)

    psi = {}
    C_g = np.empty((num_groups))

    for g in np.arange(num_groups):
        C_g[g] = len(allocation[g])   # Creates a vector of the numbers C^g
        mat = np.zeros((int(C_g[g]), J))

        for k in range(int(C_g[g])):
            mat[k, (allocation[g])[k]] = 1  # Assigns a 1 if product j is in the k'th nest in nest grouping g and zero else
        
        psi[g] = mat

    return psi


In the following, we specify some nests for the IPDL model framework. However, these aren't congruent with Figure 1 as our dataset exhibits variation of product characteristics across individuals. This means that the nesting structure should actually vary across individuals. The nests we have chosen are mostly consistent however.

In [ ]:
allocation = [[(0,1), (2,3), (4,5)], [(0, 1, 2), (3, 4, 5)]] # We specify a list of lists containing the different nests as zipped tuples of products j. 

# get nest distribution of products
psi_dict = Create_incidence_matrix(allocation, x)
G = len(allocation) # The number of nest groupings G

We have thereby obtained the incidence matrices $\Psi^1$ and $\Psi^2$ for the Fuel Type and Body Type nest groupings, respectively. For illustration, the incidence matrix $\Psi^1$ for the Fuel Type grouping contains the $C_1=3$ nests 'ev', 'cng', and 'methanol' which partition the $J=6$ cars $\{0,1, \ldots , 5\}$ into the given nests. 
Therefore $\Psi^1$ is a $3$ by $6$ matrix and is given by:

In [ ]:
pd.DataFrame(psi_dict[0], index=['ev', 'cng', 'methanol']).rename_axis(index='nests', columns='products')

# Max-rescaling for numerical stability

Let $\alpha$ be a scalar, and let $\iota$ be the all-ones vector in $\mathbb R^J$. Note that $q'(u+\alpha\iota)=q'u+(q'\iota)\alpha=q'u+\alpha$, since $q$ sums to one. For this reason, $\alpha$ does not enter into the utility maximization when calculating $P(u+\alpha\iota|\lambda)$, and we have $P(u+\alpha\iota|\lambda)=P(u|\lambda)$.

This allows us to re-scale the utilities just as in the logit model, since $P(u-(\max_{j}u_j)\iota|\lambda)=P(u|\lambda)$. The numerical benefits of this approach carry over to the IPDL model.

## Gradient and Hessian

For purposes of computing the gradient and Hessian of $\Omega$, it is convenient to define
$$
\Gamma=\left(\begin{array}{c}
(1-\sum_g \lambda_g)I_J\\
\lambda_1 \Psi^1\\
\vdots\\
\lambda_G \Psi^G
\end{array}\right)
$$
where $I_J$ is the identity matrix in $\mathbb R^J$. The matrix $\Gamma$ is a block matrix with $J+\sum_g C_g$ rows and $J$ columns. Note that 

$$
\Gamma q=\left(\begin{array}{c}
(1-\sum_g\lambda_g)q \\
\lambda_1\Psi^g q\\
\vdots \\
\lambda_G \Psi^Gq
\end{array}\right)>0
$$
if $q>0$.

Using $\Gamma$, we can show that
$$
\Omega(q|\lambda)=(\Gamma q)'\ln (\Gamma q)+c\\
\nabla_q \Omega(q|\lambda)=\Gamma'\ln (\Gamma q)+\iota\\
\nabla^2_{qq}\Omega(q|\lambda)=\Gamma'\mathrm{diag}(\Gamma q)^{-1}\Gamma,
$$
where $c$ is a scalar that depends on $\lambda$ but not on $q$ and therefore does not affect the utility maximization problem, $\iota=(1,\ldots,1)'\in \mathbb R^J$ is the all-ones vector and $\mathrm{diag}(z)$ is a diagonal matrix with the elements of the vector $z$ on the diagonal.

In [ ]:
def CreateGamma(x, Psi, Lambda):
    '''
    This function creates the Gamma matrix. 
    
    Args.
        x: a numpy matrix (N,J,K) of covariates
        Psi: a dictionary of the matrices \psi^g as columns as outputted by 'Create_incidence_matrix'
        Lambda: a numpy array (G,) of grouping parameters \lambda_g
    
    Output
        Gamma: a numpy matrix (J + sum(C_g),J) of nesting parameters times nesting distributions
    '''
    assert x.ndim == 3
    assert Lambda.ndim == 1

    N, J, K = x.shape

    Gamma_dict = {}
    for g in np.arange(len(Psi.keys())+1):
        if g == 0:
            Gamma_dict[g] =  [*((1 - np.sum(Lambda))*np.identity(J))]   # The first J rows are assigned to be diagonal matrix with 1 - sum(\lambda_g) along the diagonal
        else:
            Gamma_dict[g] = [*(Lambda[g-1]*Psi[g-1])] # The next sum(C_g) rows are assigned to be the stacked block matrices \lambda_g*\Psi^g

    Gamma = Gamma_dict[0] # We create the Gamma matrix. This command assigns Gamma to initially be the abovementioned diagonal matrix.
    
    if len(Gamma_dict.keys()) > 1:
        for g in np.arange(1,len(Gamma_dict.keys())):
            Gamma = np.concatenate((Gamma, Gamma_dict[g]), axis=0) # Stacks each of the columns of the dictionary 'Gamma_dict' i.e. the \lambda_g*\Psi^g on top of each other.

    return Gamma

We now create the $\Gamma$ matrix from our nest structure, and set the initial grouping parameters such that $\hat \lambda^0 = \left(\frac{1}{10},\frac{1}{10}\right)'$. Note that $\hat \lambda^0 > 0$ and $\sum_g \hat \lambda_g^0 = \frac{1}{5} < 1$ such that $\Omega(\cdot | \hat \lambda^0)$ is strictly convex.

In [ ]:
lambda_0 = 0.1*np.ones((G,)) # set the initial lambda parameters to be one tenth for each nest grouping.
gamma = CreateGamma(x, psi_dict, lambda_0)
pd.DataFrame(gamma, index=[*['Identity' for i in range(J)], *['Fuel type group' for i in range(psi_dict[0].shape[0])], *['Body type group' for i in range(psi_dict[1].shape[0])]]).rename_axis(index='nest groupings', columns='products')

We remark that $\Gamma$ is a $11$ by $6$ matrix since $J + C_1 + C_2 = 6 + 3 + 2 =11$ in the above example. 

For later use we also set $\hat \theta^0 = \left({ \hat \beta^{\text{logit}}}', {\lambda^{0}}' \right)'$:

In [ ]:
logit_theta = np.array([*logit_beta, *lambda_0])

## Model solution

While it is possible to solve for the choice probabilities explicitly by maximizing utility, Fosgerau and Nielsen (2021) suggest a contraction mapping approach which is conceptually simpler. Suppose we are evaluating the likelihood at some guess of the parameters $\theta=(\beta',\lambda')$. Let $u_i=\mathbf{X}_i\beta$, and let $q_i^0$ denote some initial vector of choice probabilities e.g. $q^0_i=\frac{e^{u_i}}{\sum_{j'=1}^Je^{u_{ij'}}}$, we update the choice probabilities according to the formula
$$
v_i^{k} =u_i+\ln q_i^{k-1}-\Gamma'\ln (\Gamma q_i^{k-1})\\
q_i^{k} = \frac{e^{v_i^{k}}}{\sum_{j=1}^J e^{v_{ij}^{k}}},
$$
they show that $\lim_{k\rightarrow \infty}q_i^k=p(\mathbf{X}_i,\theta)$ for any starting value $q^0_i$ in the interior of $\Delta$. For numerical stability, it can be a good idea to also do max-rescaling of $v^k_i$ at every iteration.

Let $p$ denote the solution to the utility maximization problem. Formally, the Kullback-Leibler divergence $D_{KL}(p||q)=p'\ln \frac{p}{q}$ decays linearly with each iteration,
$$
D_{KL}(p||q^{k+1})\leq (\sum_g \lambda_g)D_{KL}(p||q^k),
$$
Noting that $(1-\sum_g \lambda_g)\in [0,1)$ by assumption.

In [ ]:
def IPDL_ccp(Beta, x, Gamma, tol = 1.0e-15, maximum_iterations = 1000, MAXRESCALE:bool = True):
    ''' 
    This function finds approximations to the true conditional choice probabilities given parameters.

    Args.
        Beta: a numpy array (K,) of parameters
        x: a numpy matrix (N,J,K) of covariates
        Gamma: a numpy matrix (J + sum(C_g),J) of nesting parameters times nesting distributions as outputted by 'CreateGamma'
        tol: tolerated approximation error
        maximum_iterations: a no. of maximum iterations which if reached will stop the algorithm
        MAXRESCALE: whether or not to max rescale the choice probabilities during iteration

    Output
        q_1: a numpy matrix (N,J) of approximative IPDL choice probabilities
    '''

    u = logit.util(Beta, x)   # Find deterministic utility
    q_0 = logit.logit_ccp(Beta, x)    # Find logit choice probabilities
    q = q_0

    assert u.ndim == 2
    assert q.ndim == 2
    assert Gamma.ndim == 2

    for k in np.arange(maximum_iterations):

        # Calculate gamma*log(gamma*q^(k-1))
        gamma_q = np.einsum('cj,nj->cn', Gamma, q) # First we calculate Gamma*q^(k-1)
        log_gamma_q = np.log(gamma_q) # Then we calculate log(Gamma*q^(k-1))
        gamma_log_prod = np.einsum('cj,cn->nj', Gamma, log_gamma_q) # Finally we multiply Gamma and log(Gamma*q^(k-1)) 

        # Calculate indirect utilities
        v = u + np.log(q) - gamma_log_prod

        if MAXRESCALE:
            v -= v.max(axis=1, keepdims=True) # Max rescaling to improve numerical stability

        # Calculate iterated ccps
        denom = np.exp(v).sum(axis=1, keepdims = True) # (N,1)
        q_1 = np.exp(v) / denom

        # Check convergence in an appropriate distance function
        dist = np.max(np.sum((q_1-q)**2/q_0 , axis=1)) # Uses logit weights. This avoids precision issues when q1~q~0.

        if dist<tol:
            break
        elif k==maximum_iterations:
            break
        else:
            None
        # Iteration step
        q = q_1
    
    return q_1

## FKN estimation algorithm

The FKN estimator begins with a nonparametric estimate of the CCP function, yielding choice probabilities $\hat q^0_i$ for $i=1,\ldots,N$. We wish to find parameters such that the PUM first-order condition is approximately satisfied at $\hat q^0_i$, i.e. $\hat \theta^0$ such that 
$$
\hat q_i^0\approx P(X_i,\hat \theta^0),
$$
approximately over the sample. We introduce a residual,
$$
\hat \varepsilon^0_i(\theta)=\hat D^0_i(u(X_i,\beta)- \nabla_q \Omega(\hat q_i^0|\lambda)),
$$
where $\hat D^0_i=\textrm{diag}(\hat q^0_i)-\hat q^0_i (\hat q^0_i)'$. We have already seen that we can write
$$
u(X_i,\beta)-\nabla_q \Omega(\hat q_i^0|\lambda)=\hat G^0_i \theta-\ln \hat q^0_i
$$
where $\hat G^0_i=[X_i,\hat Z_i^0]$. Letting $\hat A^0_i=\hat D^0_i \hat G^0_i$ and $\hat r_i^0=\hat D^0_i \ln \hat q^0_i$, we therefore have
$$
\hat \varepsilon^0_i(\theta)=\hat A^0_i\theta-\hat r^0_i.
$$
We minimize the weigthed mean of these residuals over $\theta$, with weights $\hat W^0_i=\textrm{diag}(\hat q^0_i)^{-1}$,

$$
\hat \theta^0 =\arg \min_{\theta} \frac{1}{N}\sum_i \hat \varepsilon_i(\theta)'\hat W_i^0 \hat \varepsilon_i(\theta)
$$
which has the closed-form solution
$$
\hat \theta^0 =\left(\frac{1}{N}\sum_i  (\hat A^0_i)'\hat W^0_i \hat A^0_i \right)^{-1}\left(\frac{1}{N}\sum_i  (\hat A^0_i)'\hat W^0_i \hat r_i^0 \right)
$$


In [ ]:
def Z_array(psi,q):
    ''' 
    This function calculates the \hat Z matrix i.e. the negative cross differentiated hessian of the pertubation function \Omega

    Args.
        psi: dictionary of nest groupings as outputted by 'Create_incidence_matrix'
        q: (N,J) array of choice probabilities

    Returns
        Z: (N,J,G) array of the matrices \hat Z_i
    '''

    N,J=q.shape
    G=len(psi)
    Z=np.empty((N,J,G))
    log_q=np.log(q)
    
    for g in range(G):
        Psi=psi[g]
        Psiq=np.einsum('cj,nj->nc', Psi,q) # Calculates matrix product \Psi^g * q_i.
        
        log_Psiq=np.log(Psiq) 
        Psi_log_Psiq=np.einsum('cj,nc->nj', Psi, log_Psiq) # Calculates matrix product \Psi^g * log(\Psi^g * q_i)
                
        Z[:,:,g] = log_q - Psi_log_Psiq # Assign g'th matrix of Z_{njg} onto the g'th column of Z
        
    return Z

In [ ]:
def G_array(q, x, Psi):
    ''' 
    This function calculates the G block matrix

    Args.
        q: an (N,J) array of choice probabilities
        x: an (N,J,K) array of covariates
        Psi: dictionary of nest groupings as outputted by 'Create_incidence_matrix'

    Returns
        G: a (N,J,K+G) numpy array a G matrix for each individual i
    '''
    Z = Z_array(Psi, q) # Find the Z matrix
    G = np.concatenate( (x,Z), axis=2) # Join block matrices along 3rd dimensions

    return G

In [ ]:
def WeightedLeastSquares(A,W,r):
    ''' 
    This function calculates the weighted least squares estimator \hat \theta^k and its relevant estimated standard error.

    Args.
        G: (N,J,K+G) array of the \hat G_i matrices
        W: (N,J,J) array of weights \hat W_i
        r: (N,J) array of the \hat r_i vectors

    Returns
        theta_hat: (K+G,) array of estimated parameters
        se_hat: (K+G,) array of estimated standard error of parameters 
    '''

    N = A.shape[0]
    
    AWA = np.einsum('nja,njk,nkb->ab',A,W,A,optimize=True)/N # Calculates the matrix product GWG
    AWr = np.einsum('nja,njk,nk->a',A,W,r,optimize=True)/N # Calculates the matrix product GWr
    
    theta_hat = la.solve(AWA,AWr) # Solves the system of equation GWG*\theta = GWr for \theta, which are the estimated parameters
    #theta_hat=la.lstsq(GWG,GWr,rcond=None)[0] # better numerically in some case
    
    cov_hat = la.inv(AWA)/N # Calculates the (K+G,K+G) estimated covariance matrix of parameters \theta
    se_hat = np.sqrt(np.diag(cov_hat)) # Calculates the estimated standard errors of parameters \theta
    
    return theta_hat,se_hat
    

In [ ]:
def FKN_initial(q, x, Psi):
    '''
    This function calculates the initial FKN estimate on which to iterate

    Args.
        q: an (N,J) array of choice probabilities
        x: an (N,J,K) array of covariates
        Psi: dictionary of nest groupings as outputted by 'Create_incidence_matrix'

    Returns
        theta_hat: a (K+G,) numpy array of IPDL parameters
        se_hat: a (K+G,) numpy array of estimated IPDL standard errors of parameters
    '''

    N,J,K = x.shape
    G = G_array(q, x, Psi)
    
    diag_q = np.eye(J)[None,:,:] * q[:,:,None] # Computes the diag(q) matrix. Is equivalent to np.einsum('jk,nj->njk', np.eye(J), q).
    qqT = q[:,:,None] * q[:,None,:] # Computes an outer product of q with itself. Is equivalent to np.einsum('nj,nk->njk', q, q).
    D = diag_q - qqT 
    A = np.einsum('njk,nkd->njd', D, G)  # Is of dimension (N,J,K+C). Is a matrix product.
    W = np.eye(J)[None,:,:] / q[:,:,None] # Computes inverse diag(q) matrix. Is equivalent to np.einsum('jk,nj->njk', np.eye(J), 1./q).
    r = np.einsum('njk,nk->nj', D, np.log(q)) # Calculates matrix product

    theta_hat,se_hat = WeightedLeastSquares(A,W,r) # find estimated WLS parameters using closed form solution
    
    return theta_hat,se_hat

### Nonparametric estimation of the choice probabilities

The choice probabilities are a function of the entire vector of explanatory variables $x_i=\textrm{vec}(X_i)$. We stack these into a matrix $\mathbf{X}$ of size $N\times JK$.

This is a high-dimensional estimation problem. In order to reduce the dimensionality, we do a truncated singular value decomposition, i.e. fit a matrix $\mathbf{\tilde X}$ of size $N\times D$ where $D\leq JK$ and $\mathbf{\tilde X}\approx \mathbf X$.

We then estimate a bagged logistic regression with response variable $\mathbf a=(a_i)_{i=1}^N$ and input $\mathbf{\tilde X}$. 

For further details, see the scikit learn documentation or Elements of Statistical Learning (Hastie et al 2009).


We now find initial estimates of choice probabilities $\hat q_i^0$ by implementation of the procedure described above. 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import BaggingClassifier

# Create two-dimensional array X for input into ML function

X = np.empty((N,J*K))

for i in range(N):
    X[i,:] = x[i,:,:].flatten()

# Do dimensionality reduction

pca = PCA(n_components='mle')
X_pca = pca.fit_transform(X)

logistic = LogisticRegression(random_state=0, multi_class='multinomial', max_iter=10000)


fitted = logistic.fit(X_pca, a) # Fit logit probabilities 
p_hat = fitted.predict_proba(X_pca) # Find some initial guess of choice probabilities

Based on our above estimate of choice probabilities $\hat q_i^0$ we find an initial estimate $\hat \theta_{FKN}^0$ of parameters.

In [ ]:
theta_hat,se_hat = FKN_initial(p_hat, x, psi_dict)

In [ ]:
print(np.round(theta_hat,decimals=2)) # Estimated paramters. 
print(theta_hat[K:].sum()) # Convexity check for whether sum of the lambda is less than 1

## Regularization for parameter bounds

As we see above, the least squares estimator is not guaranteed to respect the parameter bounds $\sum_g \hat \lambda_g<1$. However, we know that the logit probabilities $\hat q^{logit}_i$ returns the estimates $\hat \theta^{logit}=(\hat \beta^{logit},0,0)'$. We can construct a sequence of estimators using the mixture probabilities
$$
\hat q^{(t)}_i =(1-\alpha_t) \hat q^{logit}_i+\alpha_t \hat q_i
$$
and we know that $\hat q^{(t)}_i$ respects the parameter bounds for $\alpha_t$ sufficiently close to zero by continuity. We can then compare the likelihood values of each $\hat \theta^{(t)}$ and pick the best one. This ensures that the likelihood value of the initial estimator is at least as good as the logit solution. 

Note that the benefit of doing this is that we only ever need to do a one-dimensional grid search on the interval $[0,1]$ which is very simple. 
$$
\hat \theta^*=\arg \max_{t} \mathcal L_N(\hat \theta^{(t)})
$$


In [ ]:
def IPDL_loglikelihood(Theta, a, x, Psi):
    ''' 
    This function computes the loglikehood contribution for each individual i.
    
    Args.
        Theta: a numpy array (K+G,) of parameters of (\beta', \lambda')',
        y: a numpy array (N,J) of observed choices in onehot encoding,
        x: a numpy matrix (N,J,K) of covariates,
        Psi: a dictionary of the matrices \psi^g as columns as outputted by 'Create_incidence_matrix'

    Output
        ll: a numpy array (N,) of IPDL loglikelihood contributions
    '''

    N,J,K = x.shape
    G = len(Psi.keys())

    gamma = CreateGamma(x, Psi, Theta[K:]) # The last G parameters of theta are the nesting parameters \lambda_g
    ccp_hat = IPDL_ccp(Theta[:K], x, gamma) # The first K parameters of theta are those of \beta

    ll = np.log(ccp_hat[np.arange(N), a]) # For each individual find (the log of) the choice probability of the chosen alternative. Is an (N,) array

    return ll

In [ ]:
def LogL(Theta, y = a, x = x, Psi = psi_dict):
    ''' 
    This function calculates the mean loglikelihood contribution.

    Args.
        Theta: (K+G,) array of parameters
        y: (N,) array of observed choices
        x: (N,J,K) array of covariates
        Psi: dictionary of nest groupings as outputted by 'Create_incidence_matrix'
    '''
    return np.mean(IPDL_loglikelihood(Theta, y, x, Psi))

Implementing the grid search method we find corressponding parameters $\hat \theta^*$.

In [ ]:
num_alpha = 5
alpha_line = np.linspace(0, 1, num_alpha)
logl_alpha = np.empty((num_alpha))

theta_hat_alpha = np.empty((K+G, num_alpha))
for k in range(num_alpha): # for alpha in alpha_line:

    alpha = alpha_line[k]
    
    if alpha == 0:
        theta_hat_alpha[:,k] = logit_theta # If alpha = 0 get logit paramters
    
    else: # If alpha > 0 get parameters stemming from mixture probabilities
        q_alpha = (1-alpha) * logit_q + alpha * p_hat  # Compute mixture probabilities
        theta_hat_alpha[:,k] = FKN_initial(q_alpha, x, psi_dict)[0] # Get corresponding 'mixture' parameters

    if theta_hat_alpha[K:,k].sum() > 0.999:
        logl_alpha[k] = np.NINF # Set loglikelihood to minus infinity if constraint on lambda parameters is not satisfied
    else:
        logl_alpha[k] = LogL(theta_hat_alpha[:,k], a, x, psi_dict) # Evaluate loglikelihood at parameters
    
# Pick the best one:

best_index = np.argmax(logl_alpha) # Find the best t
best_alpha = alpha_line[best_index] # Find the best alpha 
theta_hat_star = theta_hat_alpha[:,best_index] # Find the best parameters
print(best_alpha)
# easier to see the pattern for the likelihood instead of the log
plt.plot(alpha_line, np.exp(logl_alpha)) 

In [ ]:
# we see that the estimates are meaningfully better than the logit starting values

print(np.round(IPDL_theta - logit_theta, decimals=2))
print(np.round(IPDL_theta - theta_hat_star, decimals=2))

# In fact, there is only one parameter where the logit is closer to the MLE.
print(np.abs(IPDL_theta - logit_theta) > np.abs(IPDL_theta - theta_hat_star))

## Iterated FKN estimator

The iterated estimator is as the initial one, except there is an additional term on $\hat \varepsilon$. First, we update the choice probabilities,
$$
\hat q^k_i=p(\mathbf X_i,\hat \theta^{k-1})\\
$$
Then we assign
$$
\hat D^k_i=\nabla^2_{qq}\Omega(\hat q_i^k|\hat \lambda^{k-1})^{-1}-(\hat q^k_i \hat q^k_i)'
$$
and then construct the residual
$$
\hat \varepsilon^k_i(\theta)=\hat D^k_i\left( u(x_i,\beta)-\nabla_q \Omega(\hat q_i^k|\lambda)\right) -y_i+\hat q_i^k,
$$
Which can once again be simplified as
$$
\hat \varepsilon^k_i(\theta)= \hat A_i^k \theta-\hat r^k_i,
$$
where
$$
\hat A^k_i=\hat D_i^k\hat G^k_i, \hat r_i^k =\hat D^k_i\ln \hat q_i^k-y_i
$$
and where $\hat G^k_i$ is constructed as in the initial estimator. Using the weighted least squares estimator with weights $\hat W_i^k=\textrm{diag}(\hat q^k_i)^{-1}$, we get the estimator
$$
\hat \theta^k = \arg \min_{\theta}\frac{1}{n}\sum_i \hat \varepsilon^k_i(\theta)'\hat W_i^k \hat \varepsilon^k_i(\theta).
$$
We can once again solve it in closed form as
$$
\hat \theta^k =\left( \frac{1}{n}\sum_i \hat (A^k_i)'\hat W_i^k \hat A^k_i)\right)^{-1}\left( \frac{1}{n}\sum_i (\hat A_i^k)'\hat W_i^k \hat r_i^k\right)
$$
Now we implement this procedure and iterate starting from our initial guess $\hat \theta^{*}$


In [ ]:
def FKN_Iterated(theta, y, x, psi):
    ''' 
    This function calculates the step iterated estimate of parameters \hat \theta^k and its estimated standard error.
    Args.
        y: (N,J) array of observed choices as onehot encoding
        x: (N,J,K) array of covariates
        psi: dictionary of nest groupings as outputted by 'Create_incidence_matrix'
        theta: (K+G,) array of parameters from the previous iteration step. Note that an initial guess of parameters \hat \theta^0 must be specified.

    Returns
        theta_hat: (K+G,) array of estimated parameters for this iteration step
        se_hat: (K+G,) array of estimated standard error of parameters for this iteration step
    '''

    Gamma = CreateGamma(x, psi, theta[K:])
    q = IPDL_ccp(theta[:K], x, Gamma)
    
    Gamma_q = np.einsum('cj,ij->ic', Gamma, q) # is equal to Gamma*q_i stacked on each other over the individuals i
    
    H = np.einsum('cj,ic,ck->ijk', Gamma, 1./Gamma_q, Gamma) 
    H_inv = np.linalg.inv(H) #
    qqT = q[:,None,:] * q[:,:,None]
    D = H_inv - qqT
    
    G = G_array(q, x, psi_dict)  # Create the block matrix [X_i, \hat Z_i]. Joins along the K and G dimensions respectively of X and Z
    
    A = np.einsum('njk,nkl->njl', D, G) # Computes A_i = D_i * [X_i,Z_i] and stacks them along the individuals dimension.
    W = 1./q[:,:,None] * np.eye(q.shape[1])[None,:,:] # Returns W[i,:,:]= diag(q)^{-1}-\iota \iota'. Is equivalent to W=np.einsum('ij,jl->ijl', 1./q, np.eye(q.shape[1]))-1 . Computes the elementwise product of (N,J) along third dimension J and (J,J) along first dimension N. Uses broadcasting of the all 1's (N,J,J) matrix. 
    Dlog_q = np.einsum('njl,nl->nj', D, np.log(q)) # Computes diag(q_i)*log(q_i) and stacks these vectors along the individuals dimension N 
    
    r = Dlog_q + y # Computes the (N,J) array of the r_i
    
    theta_hat,se_hat = WeightedLeastSquares(A,W,r)
    
    return theta_hat,se_hat

In [ ]:
# Find FKN parameters

FKN_iters = 100
THETA = np.empty((K+G,FKN_iters))
SE = np.empty((K+G,FKN_iters))
LOGL = np.empty(FKN_iters)
tol = 1.0e-4
theta_old = theta_hat_star

for k in range(FKN_iters):
    
    theta_new,se_new = FKN_Iterated(theta_old, y, x, psi_dict)
    
    THETA[:,k],SE[:,k] = theta_new,se_new
    LOGL[k] = LogL(theta_new)
    
    if la.norm(theta_new-theta_old)<tol:
        break 
        
    theta_old = theta_new

In [ ]:
THETA[:,3] - IPDL_theta

We see that iterating all the way to convergence took some time, but the parameters are almost identical to the MLE  $\hat \theta^{\text{MLE}}$ even after a few iterations.